## **Running the models using the 'modelling' package**

For running the models, we take the following steps:

> * Prepare packages, setup, data
> * Load model
> * Define hyperparameters
> * Train the model
> * Evaluate the model

Throughout the notebook, there are printing statements to clarify potential errors happening on Habrok

#### **Prepare packages, setup, data**

In [1]:
print("Starting script...")

print("Importing modelling package...")
from modelling import *
from modelling import GRU
from modelling import HGRU

print("Importing libs...")
import os
import datetime
import numpy as np
import pandas as pd
import torch as tc
import torch.nn as nn
from torch.optim import Adam
from torch.optim.lr_scheduler import ReduceLROnPlateau
from torch.utils.data import (
    DataLoader,
    SubsetRandomSampler,
    SequentialSampler
)
import seaborn as sns
import matplotlib as mpl
import matplotlib.pyplot as plt


Starting script...
Importing modelling package...

Running __init__.py for data pipeline
Modelling package initialized

Importing libs...


Use GPU when available

In [2]:
use_cuda = tc.cuda.is_available()
device = tc.device("cuda" if use_cuda else "cpu")
print("Device: ", device)

Device:  cpu


"Global" variables

In [3]:
HABROK = bool(0)                       # True if running on Habrok or external server
if HABROK:
    print("Successfully imported libraries into env")
    USER = 'habrok'
else:
    USER = 'tinus'

if USER == 'tinus':
    os.chdir(r"c:\Users\vwold\Documents\Bachelor\ICML_paper\forecasting_smog_DL\forecasting_smog_DL\src")
    MODEL_PATH = os.path.join(os.getcwd(), "models") #! TODO get these two out and to somewhere else, or gone in general
    MINMAX_PATH = "../data/data_combined/contaminant_minmax.csv"
elif HABROK:
    os.chdir(r"/home1/s4372948/thesis/modelling/")
    MODEL_PATH = os.path.join(os.getcwd(), "models")
    MINMAX_PATH = "../data/data_combined/contaminant_minmax.csv"

tc.manual_seed(34)
mpl.rcParams['figure.figsize'] = (7, 3)

N_HOURS_U = 72
N_HOURS_Y = 24
N_HOURS_STEP = 24
CONTAMINANTS = ['NO2', 'O3', 'PM10', 'PM25']
COMPONENTS = ['NO2', 'O3', 'PM10', 'PM25', 'SQ', 'WD', 'Wvh', 'dewP', 'p', 'temp']

Load in data and create PyTorch *Datasets*

In [4]:
# Load in data and create PyTorch Datasets. To tune
# which exact .csv files get extracted, change the
# lists in the get_dataframes() definition

train_input_frames = get_dataframes('train', 'u')
train_output_frames = get_dataframes('train', 'y')

val_input_frames = get_dataframes('val', 'u')
val_output_frames = get_dataframes('val', 'y')

test_input_frames = get_dataframes('test', 'u')
test_output_frames = get_dataframes('test', 'y')

print("Successfully loaded data") if HABROK else None

In [5]:
train_dataset = TimeSeriesDataset(
    train_input_frames,  # list of input training dataframes
    train_output_frames, # list of output training dataframes
    5,                   # number of dataframes put in for both
                         # (basically len(train_input_frames) and
                         # len(train_output_frames) must be equal)
    N_HOURS_U,           # number of hours of input data
    N_HOURS_Y,           # number of hours of output data
    N_HOURS_STEP,        # number of hours between each input/output pair
)
val_dataset = TimeSeriesDataset(
    val_input_frames,    # etc.
    val_output_frames,
    3,
    N_HOURS_U,
    N_HOURS_Y,
    N_HOURS_STEP,
)
test_dataset = TimeSeriesDataset(
    test_input_frames,
    test_output_frames,
    3,
    N_HOURS_U,
    N_HOURS_Y,
    N_HOURS_STEP,
)

#### **Define model**

#### **Define evaluation and plotting functions**

#### **Define training functions**

In [8]:
def init_mb_model(hp):
    """
    Inits multi-branched, or hierarchical, model,
    by calling its constructor with the hyperparameters
    """
    return hp['model_class'](hp['n_hours_u'],
                             hp['n_hours_y'],
                             hp['input_units'],
                             hp['hidden_layers'],
                             hp['hidden_units'],
                             hp['branches'],
                             hp['output_units']
                             ).to(device)


def init_main_optimizer(model, hp):
    """Initializes the optimizer for the shared layer"""
    return hp['Optimizer'](model.parameters(), lr = hp['lr_shared'], 
                           weight_decay = hp['w_decay'])


def init_branch_optimizers(model, hp):
    """Initializes the optimizers for each branch"""
    optimizers = []
    for branch in model.branches:
        optimizers.append(hp['Optimizer'](branch.parameters(), 
                                          lr = hp['lr_branch'], 
                                          weight_decay = hp['w_decay']))
    return optimizers


def init_main_scheduler(optimizer, hp):
    """Initializes the scheduler for the shared layer"""
    return hp['scheduler'](optimizer, **hp['scheduler_kwargs'])


def init_branch_schedulers(optimizers, hp):
    """Initializes the schedulers for each branch"""
    schedulers = []
    for optimizer in optimizers:
        schedulers.append(hp['scheduler'](optimizer, **hp['scheduler_kwargs']))
    return schedulers


def init_early_stopper(hp, verbose):
     """Initializes early stopping object"""
     return hp['early_stopper'](hp['patience'], verbose)


def schedulers_epoch(main_scheduler, secondary_schedulers, val_loss = None):
    """Performs a scheduler step for each scheduler"""
    main_scheduler.step(val_loss)
    for scheduler in secondary_schedulers:
        scheduler.step(val_loss)


def print_epoch_loss(epoch, train_losses, val_losses, x = 5):
    """Prints the train and validation losses per x epochs"""
    if ((epoch + 1) % x == 0) or (epoch == 0):
        print("Epoch: {} \tLtrain: {:.6f} \tLval: {:.6f}".format(
              epoch + 1, train_losses[epoch], val_losses[epoch]))


def training_epoch_shared_layer(model, optimizer, loss_fn, train_loader):
    """Trains the shared layer of model for one epoch"""
                                        # Start by freezing the branches;
    for param in model.branches.parameters():
        param.requires_grad_(False)

    train_loss = np.float64(0)          # Loop over the training set;
    for batch_train_u, batch_train_y in train_loader:
            batch_train_u = batch_train_u.to(device)
            batch_train_y = batch_train_y.to(device)
                                        # Do the forward pass and calculate loss;
            batch_preds = tc.cat(model(batch_train_u), dim = 2)
            batch_loss = loss_fn(batch_preds, batch_train_y)
            train_loss += batch_loss.item()

            optimizer.zero_grad()       # Do the backward pass and update the weights;
            batch_loss.backward()
            optimizer.step()
    return train_loss                   # Return new loss after one "shared layer step"


def training_epoch_branches(model, optimizers, loss_fn, train_loader):
    """Trains the branches of model for one epoch"""
                                        # Start by unfreezing the branches and
                                        # freezing the shared layer;
    for param in model.branches.parameters():
            param.requires_grad_(True)
    for param in model.shared_layer.parameters():
        param.requires_grad_(False)
    
    train_loss = np.float64(0)
    model.train()                       # Loop over the training set;
    for batch_train_u, batch_train_y in train_loader:
        batch_train_u = batch_train_u.to(device)
        batch_train_y = batch_train_y.to(device)
                                        # For every batch, loop over the branches;
        for idx, optimizer in enumerate(optimizers):
                                        # Perform the forward pass and calculate loss;
                                        # Index branch dimension
            branch_batch_loss = loss_fn(
                tc.cat(model(batch_train_u), dim = 2)[:, :, idx],
                batch_train_y[:, :, idx] 
            )
            train_loss += branch_batch_loss.item()
                                        
            optimizer.zero_grad()       # Perform the backward pass and update the weights;
            branch_batch_loss.backward()
            optimizer.step()
                                        # Finally, unfreeze the shared layer and return.
    for param in model.shared_layer.parameters():
            param.requires_grad_(True)
    return train_loss


def validation_epoch(model, loss_fn, val_loader):
    """Evaluates the model on the validation set for one epoch"""
    val_loss = np.float64(0)
    model.eval()                        # Set model to evaluation mode;
    with tc.no_grad():                  # Loop over the validation set;
        for batch_val_u, batch_val_y in val_loader:
            batch_val_u = batch_val_u.to(device)
            batch_val_y = batch_val_y.to(device)
                                        # Do the forward pass, concatenate the branch outputs,
                                        # and calculate loss;
            batch_preds = tc.cat(model(batch_val_u), dim = 2)
            val_loss += loss_fn(batch_preds, batch_val_y).item()
    return val_loss


def train_mb(hp, train_loader, val_loader, verbose = False):
    """
    Trains a MBNN by freezing the branches and shared layer alternately:
    1. Initialize the model, optimizers, schedulers and early stopper,
       and determine the loss function
    2. For each epoch:
        a. Train the shared layer
        b. Train the branches
        c. Evaluate on the validation set
        d. Perform a scheduler step
        e. Print the average train and validation losses per epoch
        f. Check if the early stopper should stop the training
    3. Return the best model, and the train and validation losses
    """
    model = init_mb_model(hp)
    main_optimizer = init_main_optimizer(model, hp)
    branch_optimizers = init_branch_optimizers(model, hp)
    shared_scheduler = init_main_scheduler(main_optimizer, hp)
    branch_schedulers = init_branch_schedulers(branch_optimizers, hp)
    early_stopper = init_early_stopper(hp, verbose)
    loss_fn = hp['loss_fn']
    shared_losses, branch_losses = [], []
    train_losses, val_losses = [], []

    for epoch in range(hp['epochs']):   # For each epoch:
                                        # save losses per epoch for both network parts
        shared_losses.append(training_epoch_shared_layer(
            model, main_optimizer, loss_fn, train_loader))
        branch_losses.append(training_epoch_branches(
            model, branch_optimizers, loss_fn, train_loader))
                                        # calculate training and validation loss
        train_loss = validation_epoch(model, loss_fn, train_loader)
        val_loss = validation_epoch(model, loss_fn, val_loader)
        schedulers_epoch(shared_scheduler, branch_schedulers, val_loss)
        
                                        # save average losses per batch for each epoch
        train_losses.append(train_loss / len(train_loader))
        val_losses.append(val_loss / len(val_loader))

        print_epoch_loss(epoch, train_losses, val_losses, 1) if verbose else None
                
        if early_stopper(val_losses[epoch], epoch, model):
            break
    return early_stopper.best_model, train_losses, val_losses, shared_losses, branch_losses

Implementation of standard $k$-fold cross-validation, configurated on training a MBNN

In [9]:
def calc_means_unequal_lists(lists):
    """
    Calculates the means of lists with unequal lengths. This is useful
    for calculating the average train and validation losses per epoch,
    which may have "early stopped" at different epoch moments.
    """
    max_len = max([len(list) for list in lists])

    for list in lists:                  # First, get lists to equal length with padding,
        if len(list) < max_len:         # then, calculate the mean over the lists
            list.extend([np.nan] * (max_len - len(list)))
    return np.nanmean(lists, axis = 0)


def get_idx_KFoldXV_expanding_window(fold, n_folds, dataset_len):
    """Calculates the indices of expanding window k-fold cross validation"""
    fold_size = dataset_len // n_folds # integer division
                                       # determine ending indices of:
    if fold == n_folds - 1:            # last fold
        train_end_idx = fold_size * fold
        val_end_idx = dataset_len
    else:                              # all other folds
        train_end_idx = fold_size * (fold + 1)
        val_end_idx = fold_size * (fold + 2)

    train_indices = list(range(0, train_end_idx))
    val_indices = list(range(train_end_idx, val_end_idx))
    return train_indices, val_indices


def get_idx_KFoldXV_sliding_window(fold, n_folds, dataset_len):
    """Calculates the indices sliding window k-fold cross validation"""
    fold_size = dataset_len // (n_folds + 1)
    remainder = dataset_len % (n_folds + 1)

    train_start_idx = fold_size * fold + min(fold, remainder)
    train_end_idx = fold_size * (fold + 1) + min(fold, remainder)
    val_start_idx = train_end_idx
    val_end_idx = val_start_idx + fold_size

    train_indices = list(range(train_start_idx, train_end_idx))
    val_indices = list(range(val_start_idx, val_end_idx))
    return train_indices, val_indices
    

def KFoldXV_MB(hp, train_dataset, verbose = True): 
    """
    Performs k-fold cross validation training on a given model:
    1. For each fold:
        a. Get the indices for the current fold
        b. Initialize the train and validation loaders
        c. Train the model on the current fold
        d. Save the train and validation losses
        e. In case it has performed well, save the "best" model
    2. Calculate the average train and validation losses per epoch
    3. Return the best model, and the train and validation losses
    """
    val_losses_kfold = []

    for fold in range(hp['k_folds']):
        print(f"\n\tFold {fold + 1}/{hp['k_folds']}") if verbose else None
                                        # get indices for the current fold
        train_indices, val_indices = get_idx_KFoldXV_sliding_window(
            fold, hp['k_folds'], train_dataset.__len__())
                                        # create the train and validation loaders,
                                        # with random sampling for the train loader
        train_loader = DataLoader(train_dataset, batch_size = hp['batch_sz'], 
                            sampler = SubsetRandomSampler(train_indices))
        val_loader = DataLoader(train_dataset, batch_size = hp['batch_sz'], 
                            sampler = SequentialSampler(val_indices))
                                        # Train new model on the current fold
        _, _, val_losses, _, _ = train_mb(hp, train_loader, val_loader, verbose)
        val_losses_kfold.append(val_losses)

    return np.mean([losses[-1] for losses in val_losses_kfold])

#### **Hyperparameter tuning**

Define an ordinary grid search through the given hyperparameter space

In [10]:
def filter_dict_by_keys(dict, secondary_dict):
    """
    Filters a dictionary by the keys of another dictionary. For example:
    dict = {'a': 1, 'b': 2, 'c': 3} and secondary_dict = {'a': 0, 'c': 0}
    returns {'a': 1, 'c': 3}
    """
    return {k: dict[k] for k in secondary_dict if k in dict}


def update_dict(dict, secondary_dict):
    """
    Updates a dictionary with the keys of another dictionary. For example:
    dict = {'a': 1, 'b': 2, 'c': 3} and secondary_dict = {'a': 0, 'c': 0}
    returns {'a': 0, 'b': 2, 'c': 0}
    """
    return {**dict, **secondary_dict}


def print_dict_vertically(d):
    """Prints a dictionary formatted vertically"""
    max_key_len = max(len(key) for key in d.keys())
    for key, value in d.items():
        print(f"{key:{max_key_len}}: {value}")


def print_dict_vertically_root(d):
    """Prints a dictionary formatted vertically"""
    max_key_len = max(len(key) for key in d.keys())
    for key, value in d.items():
        print(f"{key:{max_key_len}}: {np.sqrt(value)}")


def ensure_integers(configs):
    """Ensures the hidden_layers and hidden_units are op type int"""
    for config in configs:
        config['hidden_layers'] = int(config['hidden_layers'])
        config['hidden_units'] = int(config['hidden_units'])
    return configs

# https://stackoverflow.com/questions/798854/all-combinations-of-a-list-of-lists
def gen_configs(hp_space):
    """Helper function to initiate the configuration generator"""
    keys = list(hp_space.keys())
    values = list(hp_space.values())
    configs = [dict(zip(keys, combi))   # Generate all possible combinations, see link^ for source
                    for combi in np.array(np.meshgrid(*values)).T.reshape(-1, len(values))]
    return ensure_integers(configs)


def print_current_config(config):
    """Prints the current configuration"""
    print("CURRENT CONFIGURATION:")
    print_dict_vertically(config)
    print()


def print_end_of_grid_search(best_hp, best_val_loss):
    """Prints the results of the grid search"""
    print(f"##### Best average validation loss so far: {best_val_loss:.6f} #####")
    print("With configuration:")
    print_dict_vertically(best_hp)
    print()


def grid_search(hp, hp_space, train_dataset, verbose = False):
    """Perform a grid seach through hyperparameter space"""
    best_hp, best_val_loss = {}, np.inf
                                        # For each hyperparameter configuration:
    for config in gen_configs(hp_space):
        config_dict = update_dict(hp, config)
        if verbose:
            print_current_config(config_dict)
                                        # train model with current configuration
        current_config_loss = KFoldXV_MB(config_dict, train_dataset, verbose)
        if current_config_loss < best_val_loss:
            best_hp = config_dict.copy()
            best_val_loss = current_config_loss
        if verbose:
            print(f"Average final validation loss: {current_config_loss:.6f}")
            print_end_of_grid_search(best_hp, best_val_loss)

    if not verbose:                      # print once when not verbose
        print_end_of_grid_search(best_hp, best_val_loss)
    
    return best_hp, best_val_loss

Define hyperparameters and searchable distributions

In [11]:
# Here, all (hyper)parameters are defined. The hyperparameters are defined in
# a dictionary, which is then passed to the model and the training functions.
# The grid search is performed by generating all possible combinations of the
# hyperparameters defined in the hp_space dictionary, and then performing k-fold cross
# validation on each of these configurations. The best configuration is then returned.
# When the search is finished, comment out the hp_space dictionary and save the best found
# hyperparameters in the hp dictionary, and train the final model with these.

#! TODO voeg nieuwe constructors van Model classes toe aan hp_space en ook de model initialisatie

hp = {
    'n_hours_u' : N_HOURS_U,
    'n_hours_y' : N_HOURS_Y,

    'model_class' : HGRU,
    'input_units' : train_dataset.__n_features_in__(),
    'hidden_layers' : 4,
    'hidden_units' : 64,
    'branches' : 4,
    'output_units' : train_dataset.__n_features_out__(),

    'Optimizer' : Adam,
    'lr_shared' : 1e-3,
    'scheduler' : ReduceLROnPlateau,
    'scheduler_kwargs' : {'mode' : 'min', 'factor' : 0.1,
                          'patience' : 3, 'cooldown' : 8, 'verbose' : True},
    'w_decay' : 1e-7,
    'loss_fn' : nn.MSELoss(),

    'epochs' : 5000,
    'early_stopper' : EarlyStopper,
    'patience' : 20,
    'batch_sz' : 16,
    'k_folds' : 5,
}                                   # The lr for the branched layer(s) is calculated
                                    # based on the "power ratio" between the branched
                                    # part of the network and the shared layer, which
                                    # is *assumed* to be proportional to n_hidden_layers
hp['lr_branch'] = hp['lr_shared'] * hp['hidden_layers']

# hp_space = {
#     'hidden_layers' : [2, 3, 4, 5, 6, 7, 8],
#     'hidden_units' : [16, 32, 48, 64, 80, 96, 112, 128],
#     'w_decay' : [1e-4, 1e-5, 1e-6, 1e-7, 1e-8],
# }

#### **Training the model**

Train the model by performing a grid search and saving the optimally configurated model

In [12]:
print("Starting training now") if HABROK else None

In [13]:
current_time = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
stdout_location = f'grid_search_exe_s/exe_of_HGRU_at_{current_time}.txt'
# train_dataset_full = ConcatDataset([train_dataset, val_dataset])
#                                     If HABROK, print to external file, else print to stdout
# with PrintManager(stdout_location, 'a', HABROK):
#     print(f"Grid search execution of HGRU at {current_time}\n")
#                                     # Train on the full training set
#     model, best_hp, val_loss = grid_search(hp, hp_space, train_dataset_full, True)
#                                     # Externally save the best model
#     tc.save(model.state_dict(), f"{MODEL_PATH}\model_HGRU.pth")

#     hp = update_dict(hp, best_hp)   # Update the hp dictionary with the best hyperparameters
#     print_dict_vertically(best_hp)

Lay out model architecture with optimal hyperparameters

In [14]:
with PrintManager(stdout_location, 'a', HABROK):
    print("\nPrinting model:")
    model = HGRU(hp['n_hours_u'],
                 hp['n_hours_y'],
                 hp['input_units'],
                 hp['hidden_layers'],
                 hp['hidden_units'], 
                 hp['branches'],
                 hp['output_units'])
    print(model)


Printing model:
HGRU(
  (input_layer): GRU(10, 64, batch_first=True)
  (shared_layer): GRU(64, 64, batch_first=True)
  (branches): ModuleList(
    (0-3): 4 x Branch(
      (layers): ModuleList(
        (0): GRU(64, 16, batch_first=True)
        (1): GRU(16, 16, num_layers=3, batch_first=True)
        (2): Linear(in_features=16, out_features=1, bias=True)
      )
    )
  )
)


Train model on complete training dataset (= train + validation)

In [15]:
train_loader = DataLoader(train_dataset, batch_size = hp['batch_sz'], shuffle = True)
val_loader = DataLoader(val_dataset, batch_size = hp['batch_sz'], shuffle = False) 
                                            
                                        # Train the final model on the full training set,
                                        # save the final model, and save the losses for plotting
with PrintManager(stdout_location, 'a', HABROK):
    print("\nTraining on full training set...")
    model_final, train_losses, test_losses, shared_losses, branch_losses = \
        train_mb(hp, train_loader, val_loader, True)
    tc.save(model_final.state_dict(), f'{MODEL_PATH}\model_HGRU.pth')

df_losses = pd.DataFrame({'L_train': train_losses, 'L_test': test_losses})
df_losses.to_csv(f'{os.path.join(os.getcwd(), "final_losses")}\losses_HGRU_at_{current_time}.csv', 
                 sep = ';', decimal = '.', encoding = 'utf-8')


Training on full training set...
An exception of type <class 'KeyboardInterrupt'> occurred with value 
Traceback: <traceback object at 0x00000184E38F4C00>


KeyboardInterrupt: 

#### **Testing the model**

In [ ]:
model_final = HGRU(hp['input_units'], hp['hidden_layers'], hp['hidden_units'],
                     hp['branches'], hp['output_units'])
model_final.load_state_dict(tc.load(f"{MODEL_PATH}\model_HGRU.pth"))
print(model_final)

FileNotFoundError: [Errno 2] No such file or directory: 'c:\\Users\\vwold\\Documents\\Bachelor\\ICML_paper\\forecasting_smog_DL\\forecasting_smog_DL\\src\\models\\model_MBGRU.pth'

In [ ]:
test_loader = DataLoader(test_dataset, batch_size = hp['batch_sz'], shuffle = False) 
test_error = test_hierarchical(model_final, nn.MSELoss(), test_loader)

with PrintManager(stdout_location, 'a', HABROK):
    print()
    print("Testing MSE:", test_error)

In [ ]:
print(test_hierarchical(model_final, nn.MSELoss(), train_loader))
print(test_hierarchical(model_final, nn.MSELoss(), val_loader))
print(test_hierarchical(model_final, nn.MSELoss(), test_loader))

print("\nMSE Training set:")
print_dict_vertically(
    test_hierarchical_separately(model_final, nn.MSELoss(), train_loader, True, MINMAX_PATH)
)
print("\nMSE Validation set:")
print_dict_vertically(
    test_hierarchical_separately(model_final, nn.MSELoss(), val_loader, True, MINMAX_PATH)
)
print("\nMSE Test set:")
print_dict_vertically(
    test_hierarchical_separately(model_final, nn.MSELoss(), test_loader, True, MINMAX_PATH)
)

In [ ]:
print("\nRMSE Training set:")
print_dict_vertically_root(
    test_hierarchical_separately(model_final, nn.MSELoss(), train_loader, True, MINMAX_PATH)
)
print("\nRMSE Validation set:")
print_dict_vertically_root(
    test_hierarchical_separately(model_final, nn.MSELoss(), val_loader, True, MINMAX_PATH)
)
print("\nRMSE Test set:")
print_dict_vertically_root(
    test_hierarchical_separately(model_final, nn.MSELoss(), test_loader, True, MINMAX_PATH)
)
np.sqrt(test_hierarchical(model_final, nn.MSELoss(), test_loader, True, MINMAX_PATH))

In [ ]:
pair = 5
plot_pred_vs_gt(model_final, test_dataset, pair, 'NO2', N_HOURS_Y)
plot_pred_vs_gt(model_final, test_dataset, pair, 'O3', N_HOURS_Y)
plot_pred_vs_gt(model_final, test_dataset, pair, 'PM10', N_HOURS_Y)
plot_pred_vs_gt(model_final, test_dataset, pair, 'PM25', N_HOURS_Y)